# Simple Workflow Flow Sample

This implements the sample workflow from the SWF tutorial

In [ ]:
import boto3
import uuid
client = boto3.client('swf')

In [ ]:
workflowDomain = 'TutorialWorkflowDomain2'
workflow = 'TutorialWorkflow'
tasklist = 'TutorialTasks'

In [ ]:
############################################
## DEFINITION
############################################

In [ ]:
# To me master of your domain, you must first register it.
response = client.register_domain(
    name=workflowDomain,
    description='Tutorial workflow domain',
    workflowExecutionRetentionPeriodInDays='1'
)

print response

In [ ]:
response = client.register_workflow_type(
    domain=workflowDomain,
    name=workflow,
    version='1',
    description='Tutorial workflow',
    defaultTaskStartToCloseTimeout='NONE',
    defaultExecutionStartToCloseTimeout='900',
    defaultTaskList={
        'name': tasklist
    },
    defaultChildPolicy='TERMINATE'
)

print response

In [ ]:
activities = ['get_contact_activity','subscribe_topic_activity','wait_for_confirmation_activity',
             'send_result_activity']

for activity in activities:
    response = client.register_activity_type(
        domain=workflowDomain,
        name=activity,
        version='1',
        defaultTaskList={
            'name': tasklist
        }
    )
    
    print response

In [ ]:
############################################
## EXECUTION
############################################

In [ ]:
response = client.start_workflow_execution(
  domain=workflowDomain,
  workflowId='wf-1',
  workflowType={
    "name": 'TutorialWorkflow',
    "version": '1'
  },
  taskList={
      'name': tasklist
  },
  input=''
)

print response

In [ ]:
# Is there a task available?
newTask = client.poll_for_decision_task(
    domain=workflowDomain,
    taskList={'name': tasklist},
    identity='decider-1',
    reverseOrder=False)

print newTask

In [ ]:
events = newTask['events']
print events

In [ ]:
eventHistory = [evt for evt in newTask['events'] if not evt['eventType'].startswith('Decision')]
print eventHistory

print len(eventHistory)

if eventHistory[0] != 'WorkflowExecutionStarted':
    print 'Expected freshly started workflow'
else:
    print 'Freshly started workflow'

In [ ]:
# First decision is to start the first activity
activity_id = 'activityid-' + str(uuid.uuid4())
print 'schedule first activity', activity_id

response = client.respond_decision_task_completed(
        taskToken=newTask['taskToken'],
        decisions=[
          {
            'decisionType': 'ScheduleActivityTask',
            'scheduleActivityTaskDecisionAttributes': {
                'activityType':{
                    'name': activities[0],
                    'version': '1'
                    },
                'activityId': activity_id,
                'input': '',
                'scheduleToCloseTimeout': 'NONE',
                'scheduleToStartTimeout': 'NONE',
                'startToCloseTimeout': 'NONE',
                'heartbeatTimeout': 'NONE',
                'taskList': {'name': tasklist},
            }
          }
        ]
      )

print response

In [ ]:
response = client.count_pending_decision_tasks(
    domain=workflowDomain,
    taskList={
        'name': tasklist
    }
)

print response


response = client.count_pending_activity_tasks(
    domain=workflowDomain,
    taskList={
        'name': tasklist
    }
)

print response

In [ ]:
task = client.poll_for_activity_task(
    domain=workflowDomain,
    taskList={'name': tasklist},
    identity='worker-1'
)

if 'taskToken' not in task:
    print "Poll timed out, no new task.  Repoll"
else:
    print "New task arrived"

client.respond_activity_task_completed(
    taskToken=task['taskToken'],
    result='success'
)

print "Task Done"

response = client.count_pending_activity_tasks(
    domain=workflowDomain,
    taskList={
        'name': tasklist
    }
)

print response

In [ ]:
# New task ready for the decider
response = client.count_pending_decision_tasks(
    domain=workflowDomain,
    taskList={
        'name': tasklist
    }
)

print response

In [ ]:
# Is there a task available?
newTask = client.poll_for_decision_task(
    domain=workflowDomain,
    taskList={'name': tasklist},
    identity='decider-1',
    reverseOrder=False)

print newTask

In [ ]:
eventHistory = [evt for evt in newTask['events'] if not evt['eventType'].startswith('Decision')]
print eventHistory

print len(eventHistory)

for event in eventHistory:
    print event
    print

In [ ]:
# Second decision is to start the second activity
activity_id = 'activityid-' + str(uuid.uuid4())
print 'schedule second activity', activity_id

response = client.respond_decision_task_completed(
        taskToken=newTask['taskToken'],
        decisions=[
          {
            'decisionType': 'ScheduleActivityTask',
            'scheduleActivityTaskDecisionAttributes': {
                'activityType':{
                    'name': activities[1],
                    'version': '1'
                    },
                'activityId': activity_id,
                'input': '',
                'scheduleToCloseTimeout': 'NONE',
                'scheduleToStartTimeout': 'NONE',
                'startToCloseTimeout': 'NONE',
                'heartbeatTimeout': 'NONE',
                'taskList': {'name': tasklist},
            }
          }
        ]
      )

print response

In [ ]:
task = client.poll_for_activity_task(
    domain=workflowDomain,
    taskList={'name': tasklist},
    identity='worker-1'
)

if 'taskToken' not in task:
    print "Poll timed out, no new task.  Repoll"
else:
    print "New task arrived"

client.respond_activity_task_completed(
    taskToken=task['taskToken'],
    result='success'
)

print "Task Done"

response = client.count_pending_activity_tasks(
    domain=workflowDomain,
    taskList={
        'name': tasklist
    }
)

print response

In [ ]:
# Is there a task available?
newTask = client.poll_for_decision_task(
    domain=workflowDomain,
    taskList={'name': tasklist},
    identity='decider-1',
    reverseOrder=False)

In [ ]:
eventHistory = [evt for evt in newTask['events'] if not evt['eventType'].startswith('Decision')]

for event in eventHistory:
    print event
    print